In [1]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as K
import matplotlib.pyplot as plt

K.backend.set_floatx('float64')

In [0]:
# Build the NN model
network = K.Sequential()
network.add(K.layers.Dense(16, 'tanh'))
network.add(K.layers.Dense(16, 'tanh'))
network.add(K.layers.Dense(16, 'tanh'))
network.add(K.layers.Dense(16, 'tanh'))
network.add(K.layers.Dense(1))

In [0]:
def initial_condition(x, p=1):
  return tf.sin(p*2*np.pi*x)

def solution_1d(t, x):
  return t*x*(1.0-x)*network(tf.concat([2*t-1,2*x-1], 1)) + initial_condition(x)

def heat_equation_1d(t, x):
  with tf.GradientTape() as tape1:
    tape1.watch(x)
    with tf.GradientTape() as tape2:
      tape2.watch([t,x])
      u = solution_1d(t, x)
    u_t, u_x = tape2.gradient(u, [t, x])
  u_xx = tape1.gradient(u_x, x)
  return u_t - u_xx

In [0]:
def loss(t, x):
  pde_loss = tf.reduce_mean(
    tf.square(heat_equation_1d(t, x)))
  return pde_loss 

In [0]:
def grad(t, x):
  with tf.GradientTape() as tape:
    loss_value = loss(t, x)
  return loss_value, tape.gradient(loss_value, network.trainable_variables)

def train(num_epochs=1001):
  optimizer = K.optimizers.Adam(learning_rate=0.01)

  t, x = np.meshgrid(np.linspace(0,1,50), np.linspace(0,1,50))
  t = tf.convert_to_tensor(np.reshape(t, (-1,1)))
  x = tf.convert_to_tensor(np.reshape(x, (-1,1)))

  # t_ic = tf.convert_to_tensor(np.zeros((100,1)))
  # x_ic = tf.convert_to_tensor(np.reshape(np.linspace(0,1,100), (100,1)))

  for epoch in range(num_epochs):
    loss_value, grads = grad(t, x)
    optimizer.apply_gradients(zip(grads, network.trainable_variables))
    if epoch % 100 == 0:
      print(f'{epoch}: loss = {loss_value}')

train()

In [0]:
# Plot the solution
def plot_solution():
  x = np.reshape(np.linspace(0,1,100), (100,1))
  for i in range(6):
    t = 0.02*i * np.ones((100,1))
    u = solution_1d(t, x).numpy()
    plt.plot(x, u)

  plt.plot(x, initial_condition(x), 'k--')

plot_solution()